# COMP4160_Yinjie_Liu_20211091_Assignment2_1

In this assignment, I am going to scrape the data about product review from the wesite page below:  
[Product Reviews Archive](http://mlg.ucd.ie/modules/python/assign2/20211091/)

And I will save this data into proper format for the next processes.  
In the task 2, I firstly process the data including deleting unnecessary comlumns, adding labels based on the data's info and so on....  
I tried the different models to classify two different tasks(Review Sentiment Classification and Review Helpfulness Classification). And it will be illustrated in detail in the another notebook.

In [1]:
import urllib.request
import bs4
import pandas as pd
import numpy as np
import re

# Task 1. Data Collection

The data on the webpage contains review information of different years and months,  
so I set up multiple iterations, including reading the data of the first webpage in a certain month,  
and then reading the data of other pages according to the data of the first webpage.

In [3]:
years = [2016,2017,2018,2019,2020,2021]
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

# These lists will save different types of inforamtion.
image_list = []
title_list = []
helpfulness_list = []
review_list = []

for year in years:
    for month in months:
        
        # Reading the first page data
        link = "http://mlg.ucd.ie/modules/python/assign2/20211091/reviews-" + str(year) + "-" + month +"-01.html"
        response = urllib.request.urlopen(link) 
        html = response.read().decode()

        parser = bs4.BeautifulSoup(html,"html.parser")
        
        # save the star rating of the review data
        for content in parser.find_all("img"): 
            image_src = content.get('src')
            if int(re.findall(r"\d",image_src.split("/")[-1])[0])>3:
                image_list.append(1) # positive as the number of star is bigger than 3
            else:
                image_list.append(0) # negative as the contrary condition
            
        # save the title text of the review data
        for content in parser.find_all("h5"):
            title = content.get_text() 
            title_list.append(title)
        
        # save the main body text of the review data
        for content in parser.find_all("p", class_ = 'review-body'):
            text = content.get_text()

            review_list.append(text)
        
        # save the review helpfulness information data
        for content in parser.find_all("p", class_ = 'metadata'):
            text = content.get_text()
            if "users" in text:
                helpfulness_list.append(text)
            else:
                pass
        
        
        # starting reda the rest of pages' data
        for page in range(2, int(parser.find("h4").get_text().split(" ")[-1])+1):
            link = "http://mlg.ucd.ie/modules/python/assign2/20211091/reviews-" + str(year) + "-" + month +"-0" + str(page) + ".html"
            
            response = urllib.request.urlopen(link) 
            html = response.read().decode()

            parser = bs4.BeautifulSoup(html,"html.parser")
            
            for content in parser.find_all("img"): 
                image_src = content.get('src')
                if int(re.findall(r"\d",image_src.split("/")[-1])[0])>3:
                    image_list.append(1) # positive
                else:
                    image_list.append(0) # negative
                    
            for content in parser.find_all("h5"):
                title = content.get_text() 
                title_list.append(title)

            for content in parser.find_all("p", class_ = 'review-body'):
                text = content.get_text()

                review_list.append(text)

            for content in parser.find_all("p", class_ = 'metadata'):
                text = content.get_text()
                if "users" in text:
                    helpfulness_list.append(text)
                else:
                    pass


I saved the data as dataframe into a csv file since it will be more convenient for the next work.

In [4]:
df = pd.DataFrame(list(zip(image_list,title_list,review_list, helpfulness_list)), columns = ['Label','Title', 'Review', 'Helpfulness info'])

df.to_csv("data.csv",encoding="utf_8_sig")